In [1]:
import pandas as pd
import bson
from collections import defaultdict

##### Entities

In [2]:
with open('entities.bson','rb') as f:
    ent_data = bson.decode_all(f.read())

In [102]:
print (one_supervises_two['ej8eg8lUv1W'])

set()


In [103]:
print (one_manages_two['ej8eg8lUv1W'])

set()


In [104]:
print (one_contains_two['ej8eg8lUv1W'])

set()


In [107]:
one_supervises_two = defaultdict(set)
one_contains_two = defaultdict(set)
one_manages_two = defaultdict(set)

for entry in ent_data:
    if 'position_nodes' in entry.keys():
        for node in entry['position_nodes']:
            if 'incident_edges' in node.keys():
                for ie in node['incident_edges']:
                    if 'relationship' in ie.keys() and \
                        'to' in ie.keys() and \
                        'ffrom' in ie.keys():
                        relationship = ie['relationship']
                        to_ = ie['to']
                        from_ = ie['ffrom']
                        
                        if relationship == 'supervises':
                            one_supervises_two[from_].add(to_)
                        elif relationship == 'contains':
                            one_contains_two[from_].add(to_)
                        elif relationship == 'manages':
                            one_manages_two[from_].add(to_)
                        else:
                            print ("No relationship!")

print ("Supervisor relationships: " + str(len(one_supervises_two)))
print ("Contains relationships: " + str(len(one_contains_two)))
print ("Manages relationships: " + str(len(one_manages_two)))

Supervisor relationships: 374
Contains relationships: 915
Manages relationships: 151


In [108]:
def FindRelationship(code1, code2):
    if code1 in one_supervises_two.keys():
        if code2 in one_supervises_two[code1]:
            return 'down'
    if code1 in one_manages_two.keys():
        if code2 in one_manages_two[code1]:
            return 'down'
    if code2 in one_supervises_two.keys():
        if code1 in one_supervises_two[code2]:
            return 'up'
    if code2 in one_manages_two.keys():
        if code1 in one_manages_two[code2]:
            return 'up'
    else:
        return 'same'


In [69]:
email_to_position_id = defaultdict()

for entry in ent_data:
    if 'email_address' in entry.keys() and 'position_nodes' in entry.keys():
        email = entry['email_address']
        for node in entry['position_nodes']:
            if 'uid' in node.keys():
                email_to_position_id[email] = node['uid']
                
                if 'email_addresses' in entry.keys():
                    for email in entry['email_addresses']:
                        email_to_position_id[email] = node['uid']

##### Emails

In [13]:
with open('emails.bson','rb') as f:
    email_data = bson.decode_all(f.read())

In [43]:
employee_uid_to_email = defaultdict(set)
i = 0
for email in email_data:
    if 'header_info' in email.keys():
            for header in email['header_info']:
                if 'uid' in header.keys():
                    employee_uid_to_email[header['uid']].add(header['email_address'])


In [109]:
i = 0
j = 0
employee_uid_to_position_code = defaultdict()
for uid in employee_uid_to_email:
    found = None
    not_found = []
    for email in employee_uid_to_email[uid]:
        try:
            employee_uid_to_position_code[uid] = email_to_position_id[email]
            found = email_to_position_id[email]
            i += 1
        except:
            not_found.append(email)
    if not found:
        j += 1
print (i)
print (j)
found

6116
91150


In [82]:
len(email_to_position_id)
len(employee_uid_to_email)

92664

In [145]:
email_label = {}
for i, email in enumerate(email_data):
    try:
        eid1 = email['from']
        eid2_list = email['to']
        
        u_count = 0
        n_count = 0
        s_count = 0
        d_count = 0

        for eid2 in eid2_list:
            if eid2 != eid1:
                code1 = employee_uid_to_position_code[eid1]
                code2 = employee_uid_to_position_code[eid2]
                #print (FindRelationship(code1, code2))
                if 'up' == FindRelationship(code1, code2):
                    u_count += 1
                if 'down' == FindRelationship(code1, code2):
                    d_count += 1
                if 'same' == FindRelationship(code1, code2):
                    s_count += 1  
                else:
                    n_count += 1  
        
        if u_count > d_count and u_count > 0:
            email_label[i] = 'up'
        elif d_count > u_count and d_count > 0:
            email_label[i] = 'down'
        elif s_count >= n_count:
            email_label[i] = 'same'
        else:
            email_label[i] = 'N/A'
        
    except:
        email_label[i] = 'N/A'
#print ("# Up: " + str(u_count))
#print ("# Down: " + str(d_count))
#print ("# Same: " + str(s_count))
#print ("# Missing: " + str(n_count))

In [ ]:
#formed_Set[(from_, to_)] = (contents, label)
formed_set = m

for email in email_data:
    

In [146]:
with open('threads.bson','rb') as f:
    thread_data = bson.decode_all(f.read())

In [148]:
email_uid_to_index = defaultdict(int)
for i, email in enumerate(email_data):
    email_uid_to_index[email['uid']] = i

In [149]:
thread_with_content = defaultdict(list)

for thread in thread_data:
    if 'thread_nodes' in thread.keys():
        for node in thread['thread_nodes']:
            if 'message_id' in node.keys():
                thread_with_content[thread['uid']].append(email_data[email_uid_to_index[node['message_id']]])
        

In [150]:
i = 0
for email in thread_with_content:
    for i in range(len(thread_with_content[email])):
        #print (thread_with_content[email][i]['uid'])
        label = email_label[email_uid_to_index[thread_with_content[email][i]['uid']]]
        if label != 'N/A':
            print (str(email) + ' ' + label)
    
    #em = email_uid_to_index[email]
    #print (email_label[em])
    i+=1
    if i > 1000:
        break
    

4238 same
4238 same
4240 down
4275 same
4275 same
4305 same
16249 same
4335 same
4365 same
4377 same
4377 same
4399 same
4399 same
4399 same
4418 same
4463 down
4497 same
4506 down
4518 same
16283 same
4545 same
4557 down
4561 same
4576 same
4576 same
4623 same
4627 same
4637 same
4637 same
4689 same
4709 same
4709 same
4715 same
4715 same
4753 same
4753 same
4754 same
4754 same
4754 same
4779 same
4911 same
4911 same
4917 same
4919 same
4919 same
4923 same
4938 same
4947 same
4960 same
4973 same
4973 same
4992 same
4992 same
4992 same
4992 same
4998 same
4998 down
5034 same
5059 same
5075 same
16290 same
16290 same
16300 same
16300 same
16307 same
16351 same
16394 same
16394 up
16404 same
16416 same
16427 same
5156 same
5156 same
5156 same
5206 same
5232 same
5232 same
5244 same
5244 same
5247 same
5247 same
5256 same
5266 same
5267 same
5317 same
5380 same
5397 same
5406 same
5406 same
5459 same
5478 same
5532 same
5532 same
16473 same
16473 same
22110 same
5542 same
16526 same
22123

28418 same
28442 same
28442 same
28442 same
28458 same
28461 same
28472 same
28474 same
28484 same
28485 same
28485 same
28493 same
28493 same
33426 same
33426 down
43668 same
43672 same
43672 same
43672 same
43677 up
43677 same
43677 same
43685 same
43685 same
43685 same
43685 same
43694 same
76065 same
76067 same
33446 same
7063 same
7063 same
7073 same
7073 same
7074 same
7074 same
7075 same
7075 same
7091 same
7091 same
7092 same
7092 same
17399 same
17399 same
17403 same
17403 same
17403 same
17403 same
17425 same
17437 down
43723 same
43733 same
43744 same
43744 same
43744 same
43765 same
43765 same
43801 same
43809 same
43809 same
43866 same
43868 same
43868 same
43875 same
43875 same
43876 same
43876 same
43876 same
43900 same
43929 same
43929 same
43929 same
43933 same
43933 same
43947 same
38472 same
93325 same
93343 same
7118 same
7118 same
82260 same
82260 same
43950 same
28508 same
28508 same
60178 same
54885 same
54885 same
22913 same
17445 same
82262 same
82262 same
7134

12939 same
44488 same
44488 same
44488 same
23502 same
28968 same
56025 same
39333 same
44500 same
44500 same
18154 same
23519 same
28969 same
28969 same
28969 same
34152 same
34152 same
83209 same
12950 same
49757 same
49757 same
94013 same
23523 up
44539 same
44539 same
44541 same
44541 same
56033 same
23524 same
23524 same
23524 same
113473 same
12952 same
18192 same
39337 same
61927 same
61927 same
34157 same
83217 same
76728 same
76728 same
56039 same
56039 same
56039 same
56039 same
94029 same
23549 same
7743 up
7743 same
12958 same
12962 same
49770 same
49770 same
94031 same
56052 same
56052 same
34179 same
56077 same
113477 same
94043 same
94043 same
94043 same
94043 same
44555 same
44555 same
23557 same
23557 same
34184 same
12973 down
83226 same
49774 same
34190 same
34190 same
34190 same
76750 same
76750 same
18207 same
18207 same
18207 same
61939 same
61939 same
61939 same
61939 same
61939 same
61939 same
61939 same
61939 same
61939 same
61939 same
61939 same
61939 same
940

45548 same
45548 same
45561 same
45569 same
45569 same
45569 same
45573 same
45574 same
45574 same
45582 same
45582 same
45583 same
45583 same
45622 same
45638 same
45640 same
45661 same
45661 same
45695 same
45695 same
45695 same
45695 same
45695 same
45703 same
45708 same
45726 same
45738 same
45738 same
45738 same
45739 same
45739 same
45739 same
45739 same
45739 same
45739 same
45753 same
45753 same
45753 same
45777 same
45780 same
45781 same
45781 same
45781 same
45781 same
45782 same
45783 same
45797 same
45797 same
45882 same
45890 same
45890 same
45924 same
45924 same
45925 same
45937 same
45947 same
45958 same
45958 same
45958 same
45958 same
45958 same
45969 same
45969 same
45974 same
45980 same
45980 same
45980 same
45985 same
45985 same
45986 same
46005 same
46039 same
46090 same
46130 same
46130 same
46138 same
46148 same
46148 same
46148 same
46148 same
46160 same
46161 same
46164 same
46164 same
46164 same
46167 same
46167 same
83937 same
24376 same
24376 same
56848 same

63307 same
84553 same
30427 same
30427 same
30427 same
46911 up
19756 up
19756 down
40573 same
30429 same
84561 same
25034 same
25034 same
95728 same
51271 same
19769 same
19772 same
40585 same
40585 same
78373 same
78373 same
30450 same
30462 same
63329 same
63329 same
63329 same
14295 same
46929 same
19773 same
19774 same
19783 same
19785 same
40587 down
40590 down
40594 same
40594 same
40603 same
40616 same
40626 same
40626 same
40631 same
40631 same
40670 same
40670 same
19786 same
19786 same
51273 same
51273 same
40672 same
40672 same
9380 same
57845 same
57845 same
25040 same
25040 same
57846 same
57846 same
51286 same
51286 same
19800 same
19800 same
19800 same
95736 same
95736 same
95736 same
95736 same
95736 same
95736 same
19818 same
57863 same
57863 same
30478 same
30478 same
9391 same
9391 same
95759 same
95759 same
25049 same
78401 same
84565 same
51298 same
40684 same
40684 same
40684 same
40684 same
14306 same
14306 same
78405 same
78405 same
46967 same
46967 same
46977 

...

121594 same
121604 down
151258 same
151277 same
126064 same
126064 same
126064 same
151375 same
151375 same
151392 same
151397 same
151397 same
151397 same
151464 same
151476 same
121622 same
121622 same
121622 same
121643 same
121689 same
145871 same
145871 same
145879 same
145879 same
136367 same
136371 up
136386 same
136386 same
131145 same
131145 same
131145 same
136415 same
136424 same
136424 same
136429 same
136429 same
136429 same
136438 same
110844 same
110857 same
110857 same
110857 same
111033 same
111037 same
111058 same
111075 same
111075 same
111076 same
111076 same
111076 same
145882 up
145955 same
145974 same
145985 same
145985 same
146007 down
146007 up
146017 same
146024 same
146037 same
146045 same
146045 same
111171 same
111171 same
121692 same
121692 same
121702 same
121702 same
121705 same
121707 same
121707 same
121737 same
121737 same
121744 same
141161 same
141161 same
141161 same
141164 same
121751 same
121751 same
151484 same
151484 same
131204 same
131204 sam

137146 same
3137 same
3137 same
3138 same
3138 same
3138 up
120 same
127049 same
127049 same
164373 same
164373 same
173776 same
173776 same
180425 same
152986 same
152986 same
152986 same
152986 same
147153 same
171585 same
171585 same
132377 same
132377 same
132391 same
132392 same
132392 same
132429 same
132429 same
132429 same
132455 same
132455 same
132455 same
164401 same
164401 same
164401 same
142612 same
142613 same
142613 same
142616 same
142616 same
164418 same
159165 same
159165 same
164448 same
164448 same
164467 same
164467 same
164467 same
164482 same
164491 same
164491 same
147156 same
132472 same
132472 same
132474 same
132480 up
164527 same
164527 same
164546 same
164546 same
173785 same
132495 same
132497 same
132504 same
132518 same
132519 same
132523 same
132564 same
132564 same
132577 same
132577 same
132577 same
132594 same
132594 same
132594 same
132594 same
132594 same
127063 same
127063 same
127069 same
127069 same
127073 same
127073 same
127079 same
127091 up

143575 down
166256 same
166256 same
166256 same
3297 same
133779 same
3298 same
3298 same
123609 same
166274 same
166274 same
127875 same
123613 same
123617 same
123617 same
123620 same
123620 same
123620 same
123621 same
123621 same
123627 same
3301 same
3301 same
148021 same
181291 same
181291 same
123644 same
316 same
316 same
133797 same
133797 same
133797 same
143589 same
143589 same
3302 same
3302 same
3302 same
127877 same
317 same
317 same
148027 same
127880 same
143594 same
1278 same
1278 same
1279 same
1279 same
1280 same
1280 same
160516 same
3304 same
123655 same
123655 same
138089 same
3305 same
3305 same
3306 same
3306 same
3306 same
3306 same
3306 same
3306 same
3306 same
3306 same
3306 same
3306 same
3306 same
3306 same
3306 same
3306 same
185408 same
123660 same
123660 same
143605 same
1281 same
175159 same
160539 same
138096 same
166361 same
166361 same
175169 same
175169 same
175212 same
175212 same
175212 same
133810 same
133810 same
133810 same
172624 same
1284 sam

182072 same
2374 same
2374 same
2375 same
2375 same
2375 same
2375 same
2375 same
2375 same
2375 same
2378 same
2378 same
182079 same
182079 same
1509 same
1509 up
1509 up
1509 down
182091 same
1510 same
187031 same
3525 down
139198 same
139198 same
1513 same
3526 same
3526 same
3526 same
2382 same
2382 same
2382 same
2382 same
2382 same
2382 same
2382 same
176696 same
129220 same
1514 same
3527 same
3527 same
3543 same
3543 same
139216 same
139216 same
176708 same
2384 same
540 same
176720 same
176720 same
129246 same
129246 same
129259 same
129259 same
129259 same
129300 same
129300 same
129300 same
129300 same
129300 same
182137 same
3545 same
3545 same
2386 same
2386 same
2386 same
543 same
543 same
182141 same
182141 same
182141 same
2388 same
129324 same
1521 same
1521 up
1521 down
1521 up
1521 up
1521 same
1521 same
1522 same
1522 same
129343 same
182158 same
182158 same
182158 same
182158 same
182158 same
1524 same
1525 same
176786 same
176786 same
176786 same
139246 same
13924

2997 same
2999 same
3002 same
3002 same
3003 same
3010 same
3016 same
3016 same
3016 same
3022 same
3022 same
3023 same
3024 same
3024 same
3024 same
3029 same
3035 same
3035 same
3037 same
3037 same
2057 same
2060 same
2062 same
3040 same
3041 same
3041 same
3042 same
3042 same
3042 same
3042 same
2068 same
2070 same
2070 same
2071 same
2071 same
2073 same
2073 same
2073 same
2074 same
2078 same
2078 same
2078 same
3058 same
3058 same
3067 same
3067 same
3068 same
3073 same
3080 same
3080 same
67 same
69 same
71 same
71 same
2083 same
3085 same
3085 same
3085 same
3088 same
3088 same
3088 same
77 same
84 same
84 same
86 same


##### Emails

In [ ]:
with open('emails.bson','rb') as f:
    email_data = bson.decode_all(f.read())

In [175]:
i = 0
from collections import defaultdict

#[(from_, to_)] = concat(subject + body) 
conversation_strings = defaultdict(str)
conversation_count = defaultdict(int)
conversation_labels = defaultdict(str)

for i, email in enumerate(email_data):
    to_, from_, subject_, body_ = "", "", "", ""
    
    if not 'to' in email.keys() or not 'from' in email.keys():
        continue
    if not 'subject' in email.keys() and not 'body' in email.keys():
        continue
           
    to_ = email['to']
    from_ = email['from']
    
    if from_ is None:
        continue
        
    if 'subject' in email.keys():
        subject_ = email['subject']
    if 'body' in email.keys():
        body_ = email['body']


        
    for address in to_:
        if from_ == address:
            continue
        index_ = (from_, address)
        
        conversation_labels[index_] = email_label[i]
        conversation_strings[index_] += subject_ + " " + body_
        conversation_count[index_] += 1


In [176]:
total_length = len(conversation_strings.keys())
total_count = 0
convos_count = 0

for c_ in conversation_count:
    convos_count += conversation_count[c_]

for rc in conversation_strings:
    total_count += len(conversation_strings[rc])
    
print ("Total relationships: " + str(total_length))
print ("Avg char. count of content: " + str(total_count / total_length))

print ("\nTotal e-mails sent: " + str(convos_count))
print ("Avg convo. count per relationship: " + str(convos_count / total_length))


Total relationships: 329580
Avg char. count of content: 5803.294408034468

Total e-mails sent: 1336107
Avg convo. count per relationship: 4.053968687420353


In [178]:
# Convert to label -> text
pre_prep_format = []
u_count = 0
nu_count = 0
n_count = 0
for key in conversation_strings.keys():
    
    if conversation_labels[key] == 'up':
        u_count += 1
    elif conversation_labels[key] == 'down':
        nu_count += 1
    elif conversation_labels[key] == 'same':
        n_count += 1
        
    #else:
    #    n_count += 1
    #    pre_prep_format.append(('neutral', relationship_content[key]))

In [180]:
print ("Total relationships: " + str(len(conversation_strings)))
print ("Upward messages: " + str(u_count))
print ("Not-upward messages: " + str(nu_count))
print ("Neutral messages: " + str(n_count))

Total relationships: 329580
Upward messages: 147
Not-upward messages: 89
Neutral messages: 6757


In [ ]:
# Label issues testing
i = 0
relationships = []
for key in conversation_labels.keys():
    relationships.append[key]
    i += 1
    if i > 5: break

In [181]:
import re
import nltk
import string
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [183]:
stop = set(stopwords.words('english'))

post_prep_format = []

for key in conversation_strings.keys():
    label = conversation_labels[key]
    
    if label != 'up' and label != 'down':
        continue
    message = conversation_strings[key]
    
    processed_message = message.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(processed_message)
    
    filtered_words = filter(lambda token: token not in stopwords.words('english'), tokens)
    
    post_prep_format.append((label, ' '.join(filtered_words)))

In [ ]:
#print (post_prep_format[50])

In [184]:
df = pd.DataFrame(post_prep_format, columns=['label', 'message'])

In [185]:
df_train = df[:round(df.shape[0] *.7)]
df_test = df[round(df.shape[0] *.7):]

In [186]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import numpy as np

tfidf_transformer = TfidfTransformer()
baseline_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB()),
])

In [187]:
baseline_clf.fit(df_train['message'], df_train['label'])

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [188]:
predicted = baseline_clf.predict(df_test['message'])
print ("Initial baseline accuracy: " + str(np.mean(predicted == df_test['label'])))

Initial baseline accuracy: 0.6338028169014085


In [189]:
predicted

array(['up', 'up', 'up', 'up', 'up', 'down', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'down', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up'], dtype='<U4')

In [190]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [191]:
model = Sequential()

In [192]:
df_train['message'].shape

(165,)

In [193]:
word_count = set([])

for message in df_train['message']:
    word_count.add(message)

In [194]:
len(word_count)

153

In [195]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

In [196]:
num_labels = 2
vocab_size = 15000
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df_train['message'])
 
x_train = tokenizer.texts_to_matrix(df_train['message'], mode='tfidf')
x_test = tokenizer.texts_to_matrix(df_test['message'], mode='tfidf')
 
encoder = LabelBinarizer()
encoder.fit(df_train['label'])
y_train = encoder.transform(df_train['label'])
y_test = encoder.transform(df_test['label'])

In [197]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               7680512   
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1026      
__________

In [202]:
prediction = model.predict_classes(x_test)
correct = 0
for i in range(len(prediction)):
    if prediction[i] == y_test[i]:
        correct += 1
print (correct / len(prediction))

0.8028169014084507
[1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0 1 0
 1 1 1 1 0 0 0 0 0 0 1 0 0 1 1 0 1 1 1 1 0 0 0 0 1 0 1 1 0 0 1 0 1 1]
[[1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]]
